In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
# from models.POD import PODNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
from collections import Counter

In [3]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [4]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear/non_linear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/fourier_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/fourier_model


In [5]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear/non_linear.pkl


In [6]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [8]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [9]:
def modes_base(data, n_modes):

    # FFT decomposition and obtain energy of each mode
    fft_data = torch.fft.fft2(data)
    fft_data_shifted = torch.fft.fftshift(fft_data)
    energy = torch.abs(fft_data_shifted)
    energy_flattened = energy.flatten(1, 3)

    # Get the n_modes more energetic modes and their indices
    top_energetic = torch.topk(energy_flattened, n_modes).indices
    top_energetic_indices, _ = zip(*Counter(top_energetic.flatten()).most_common(n_modes))
    top_energetic_indices = list(map(int, top_energetic_indices))

    # Create an empty template to include the modes
    filtered_modes = torch.zeros_like(energy, dtype=torch.complex64)
    filtered_modes.flatten(1, 3)[:, top_energetic_indices] = fft_data_shifted.flatten(1, 3)[:, top_energetic_indices]

    # Return the base with the 'n_modes' most energetic modes
    return top_energetic_indices, filtered_modes

def reconstruct_data(coefficients_shifted):
    
    # Compute inverse FFT and reconstruct data
    filtered_modes_base = torch.fft.ifftshift(coefficients_shifted)
    reconstructed_data = torch.real(torch.fft.ifft2(filtered_modes_base))

    return reconstructed_data

In [10]:
num_modes = 100

indices_base, _ = modes_base(data=y_train.values, n_modes=num_modes)
reconstructed_data = reconstruct_data(_)

In [11]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [12]:
import torch
import torch.nn as nn

from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My

class Encoder(nn.Module):

    def __init__(self, input_size, hidden_layer_1_size, hidden_layer_2_size, latent_space_size):
        super(Encoder, self).__init__()

        # Parameters
        self.in_size = torch.tensor(input_size)
        self.h1_size = hidden_layer_1_size
        self.h2_size = hidden_layer_2_size
        self.ls_size = latent_space_size

        # Architecture
        self.flatten_layer = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden1_layer = nn.Linear(torch.prod(self.in_size), self.h1_size)
        self.hidden2_layer = nn.Linear(self.h1_size, self.h2_size)
        self.latent_space_layer = nn.Linear(self.h2_size, self.ls_size)
    
    def forward(self, X):
        
        X = self.flatten_layer(X)
        X = torch.sigmoid(self.hidden1_layer(X))
        X = torch.sigmoid(self.hidden2_layer(X))
        latent_space_output = (self.latent_space_layer(X))

        return latent_space_output
    
class Explanatory(nn.Module):

    def __init__(self, input_size, n_filters, hidden_layer_size, output_size):
        super(Explanatory, self).__init__()

        # Parameters
        self.in_size = torch.tensor(input_size)
        self.n_filters = n_filters
        self.h_layer = hidden_layer_size
        self.out_size = torch.tensor(output_size)

        # Architecture
        self.conv_expand_layer = nn.Conv2d(in_channels=1, out_channels=self.n_filters, kernel_size=1)
        self.flatten_layer = nn.Flatten(start_dim=1, end_dim=-1)
        self.hidden_layer = nn.Linear(n_filters*torch.prod(self.in_size), n_filters*torch.prod(self.out_size))
        self.conv_converge_layer = nn.Conv2d(in_channels=n_filters, out_channels=1, kernel_size=1)
        
    def forward(self, X):
        
        X = torch.sigmoid(self.conv_expand_layer(X))
        X = self.flatten_layer(X)
        X = self.hidden_layer(X)
        X = X.view(X.size(0), self.n_filters, self.out_size[1], self.out_size[2])
        explanatory_output = self.conv_converge_layer(X)

        return explanatory_output
    
class FFTNonlinearModel(nn.Module):
    
    def __init__(self, input_size, predictive_layers, FFT_modes_base, output_predictive_size, explanatory_input_size, explanatory_layers, output_explanatory_size, n_filters):
        
        super(FFTNonlinearModel, self).__init__()

        # Parameters
        self.in_size = input_size
        self.pred_size = predictive_layers
        self.out_pred_size = output_predictive_size
        
        self.in_exp_size = explanatory_input_size
        self.exp_size = explanatory_layers
        self.out_exp_size = output_explanatory_size

        self.n_filters = n_filters

        # Architecture
        self.encoder = Encoder(self.in_size, self.pred_size[0], self.pred_size[1], 2*self.pred_size[2])
        self.base_indices = FFT_modes_base
        self.explanatory = Explanatory(self.in_exp_size, self.n_filters, self.exp_size[0], self.out_exp_size)
        
    def forward(self, X):

        # Predictive network
        X = self.encoder(X)

        # Manipulating output to obtain real and complex part
        output_predictive = X.view(X.size(0), self.pred_size[2], 2)
        real = output_predictive[..., 0]
        imag = output_predictive[..., 1]
    
        # Reconstruction with FFT and manipulation of prediction output
        base = torch.zeros((X.size(0), *self.out_pred_size), dtype=torch.complex64).to(DEVICE)
        base.flatten(1, 3)[:, self.base_indices] = torch.complex(real, imag)

        u = reconstruct_data(base).to(DEVICE)        
        um = Mx(My(TensOps(u, space_dimension=2, contravariance=0, covariance=0))).values

        # Explanatory network
        K = self.explanatory(um)
        
        return u, K

In [13]:
# Load model and the optimizer
model = FFTNonlinearModel(input_shape, predictive_layers, indices_base, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 20000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 8.477e+08, Test loss: 1.077e+09, MSE(e): 8.474e+01, MSE(pi1): 2.331e+01, MSE(pi2): 3.413e+01, MSE(pi3): 3.005e-01
Epoch 100, Train loss: 6.649e+08, Test loss: 8.340e+08, MSE(e): 6.648e+01, MSE(pi1): 8.822e-01, MSE(pi2): 2.673e+01, MSE(pi3): 1.838e-01
Epoch 200, Train loss: 4.876e+08, Test loss: 6.207e+08, MSE(e): 4.875e+01, MSE(pi1): 5.801e-01, MSE(pi2): 1.968e+01, MSE(pi3): 1.359e-01
Epoch 300, Train loss: 3.649e+08, Test loss: 4.718e+08, MSE(e): 3.649e+01, MSE(pi1): 5.757e-01, MSE(pi2): 1.478e+01, MSE(pi3): 1.325e-01
Epoch 400, Train loss: 2.721e+08, Test loss: 3.580e+08, MSE(e): 2.721e+01, MSE(pi1): 5.756e-01, MSE(pi2): 1.106e+01, MSE(pi3): 1.325e-01
Epoch 500, Train loss: 2.017e+08, Test loss: 2.704e+08, MSE(e): 2.017e+01, MSE(pi1): 5.756e-01, MSE(pi2): 8.249e+00, MSE(pi3): 1.325e-01
Epoch 600, Train loss: 1.490e+08, Test loss: 2.039e+08, MSE(e): 1.490e+01, MSE(pi1): 5.757e-01, MSE(pi2): 6.141e+00, MSE(pi3): 1.325e-01
Epoch 700, 

In [14]:
# Parametros de entrenamiento
start_epoch = 18000
n_epochs = 30000

batch_size = 64
n_checkpoints = 100

second_lr = None

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 18000.
Epoch 18000, Train loss: 1.194e+05, Test loss: 1.303e+08, MSE(e): 1.183e-02, MSE(pi1): 6.554e-02, MSE(pi2): 7.527e-03, MSE(pi3): 4.168e-03
Epoch 18100, Train loss: 7.554e+04, Test loss: 1.305e+08, MSE(e): 7.487e-03, MSE(pi1): 3.323e-02, MSE(pi2): 5.639e-03, MSE(pi3): 3.330e-03
Epoch 18200, Train loss: 7.488e+04, Test loss: 1.307e+08, MSE(e): 7.383e-03, MSE(pi1): 6.205e-02, MSE(pi2): 5.557e-03, MSE(pi3): 4.163e-03
Epoch 18300, Train loss: 7.368e+04, Test loss: 1.308e+08, MSE(e): 7.296e-03, MSE(pi1): 3.793e-02, MSE(pi2): 5.494e-03, MSE(pi3): 3.406e-03
Epoch 18400, Train loss: 7.493e+04, Test loss: 1.309e+08, MSE(e): 7.410e-03, MSE(pi1): 4.709e-02, MSE(pi2): 5.484e-03, MSE(pi3): 3.617e-03
Epoch 18500, Train loss: 8.057e+04, Test loss: 1.311e+08, MSE(e): 7.986e-03, MSE(pi1): 3.761e-02, MSE(pi2): 5.881e-03, MSE(pi3): 3.220e-03
Epoch 18600, Train loss: 7.742e+04, Test loss: 1.313e+08, MSE(e): 7.658e-03, MSE(pi1): 5.321e-02, MSE(pi2): 5.687e-0

In [15]:
# Parametros de entrenamiento
start_epoch = 29700
n_epochs = 50000

batch_size = 64
n_checkpoints = 100

second_lr = None

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
           D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 29700.


Epoch 29700, Train loss: 3.493e+04, Test loss: 1.311e+08, MSE(e): 3.336e-03, MSE(pi1): 1.238e-01, MSE(pi2): 2.632e-03, MSE(pi3): 3.229e-03
Epoch 29800, Train loss: 4.117e+04, Test loss: 1.311e+08, MSE(e): 4.064e-03, MSE(pi1): 2.317e-02, MSE(pi2): 3.055e-03, MSE(pi3): 2.893e-03
Epoch 29900, Train loss: 3.430e+04, Test loss: 1.311e+08, MSE(e): 3.363e-03, MSE(pi1): 3.859e-02, MSE(pi2): 2.625e-03, MSE(pi3): 2.854e-03
Epoch 30000, Train loss: 4.779e+04, Test loss: 1.310e+08, MSE(e): 4.723e-03, MSE(pi1): 2.588e-02, MSE(pi2): 3.225e-03, MSE(pi3): 3.002e-03
Epoch 30100, Train loss: 3.860e+04, Test loss: 1.311e+08, MSE(e): 3.757e-03, MSE(pi1): 7.406e-02, MSE(pi2): 2.853e-03, MSE(pi3): 2.846e-03
Epoch 30200, Train loss: 5.925e+04, Test loss: 1.313e+08, MSE(e): 5.850e-03, MSE(pi1): 4.385e-02, MSE(pi2): 3.892e-03, MSE(pi3): 3.081e-03
Epoch 30300, Train loss: 3.675e+04, Test loss: 1.311e+08, MSE(e): 3.613e-03, MSE(pi1): 3.221e-02, MSE(pi2): 2.709e-03, MSE(pi3): 2.945e-03
Epoch 30400, Train loss: 3.